## 참고자료

### 아래 크롤링 코드 : https://yobbicorgi.tistory.com/31?category=478264

#### 진행 중 참고 사항 : 
- [파이썬] selenium 셀레니움 설치하기 / 크롬 드라이버 설치하기 https://0433.tistory.com/40  

- 크롬드라이버 설치 페이지
https://chromedriver.chromium.org/downloads

- [PYTHON] selenium webdriver 경로 설정하기
https://emessell.tistory.com/148

- MAC OS 개발자를 확인할 수 없기 때문에 chromedriver를 열수없습니다 해결 방법https://somjang.tistory.com/entry/MAC-OSX-%EA%B0%9C%EB%B0%9C%EC%9E%90%EB%A5%BC-%ED%99%95%EC%9D%B8%ED%95%A0-%EC%88%98-%EC%97%86%EA%B8%B0-%EB%95%8C%EB%AC%B8%EC%97%90-chromedriver%EC%9D%84%EB%A5%BC-%EC%97%B4-%EC%88%98-%EC%97%86%EC%8A%B5%EB%8B%88%EB%8B%A4-%ED%95%B4%EA%B2%B0-%EB%B0%A9%EB%B2%95

- txt 파일읽기 에러해결 https://newpower.tistory.com/123

- 크롤링 진행 시 검색버튼 클릭 안될 때
https://wkdtjsgur100.github.io/selenium-does-not-work-to-click/

- 로딩될 때까지 크롤링 시간 조절
https://charimlab.tistory.com/entry/ep01%EC%9B%B9%ED%81%AC%EB%A1%A4%EB%A7%81-17-%EC%85%80%EB%A0%88%EB%8B%88%EC%9B%80-%EB%A1%9C%EB%94%A9%EB%90%A0-%EB%95%8C%EA%B9%8C%EC%A7%80-%EC%8B%9C%EA%B0%84%EC%A1%B0%EA%B1%B4-%EC%84%A4%EC%A0%95%ED%95%98%EA%B8%B0-with-%ED%8C%8C%EC%9D%B4%EC%8D%AC


In [1]:
pip install selenium

You should consider upgrading via the '/Users/sujin/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [8]:
import os
os.getcwd()

'/Users/sujin/Desktop/멋사/Team Project'

In [52]:
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
import urllib.request
from multiprocessing import Pool
import pandas as pd


key=pd.read_csv('./keyword.txt',encoding='utf-8',names=['keyword']) ##동일 디렉토리에 검색 키워드 리스트업하여 keyword.txt파일로 저장
keyword=[]
[keyword.append(key['keyword'][x]) for x in range(len(key))]


def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

def image_download(keyword):
    createFolder('./'+keyword+'_high resolution')
    
    driver = webdriver.Chrome(r"/Users/sujin/Desktop/멋사/Team Project/chromedriver") ##크롬 웹드라이버 설치 위치 
    driver.implicitly_wait(3)
    
    print(keyword, '검색')
    driver.get('https://www.google.co.kr/imghp?hl=ko')
    
    Keyword=driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input')
    Keyword.send_keys(keyword)
    
    #driver.find_element_by_xpath('//*[@id="sbtc"]/button').click()
    driver.find_element_by_xpath('//*[@id="sbtc"]/button').send_keys(Keys.ENTER)

    print(keyword+' 스크롤 중 .............')
    elem =  driver.find_element_by_tag_name("body")
    for i in range(60):
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.1)
        
    try:
        driver.find_element_by_xpath('//*[@id="islmp"]/div/div/div/div[1]/div[4]/div[2]/input').click()
        for i in range(60):
            elem.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.1)
    except:
        pass
    
    images = driver.find_elements_by_css_selector("img.rg_i.Q4LuWd")
    print(keyword+' 찾은 이미지 개수:',len(images))
    
    links=[]
    for i in range(1,len(images)):
        try:
            driver.find_element_by_xpath('//*[@id="islrg"]/div[1]/div['+str(i)+']/a[1]/div[1]/img').click()
            #driver.find_element_by_xpath('//*[@id="islrg"]/div[1]/div['+str(i)+']/a[1]/div[1]/img').send_keys(Keys.ENTER)
            
            #links.append(driver.find_element_by_xpath('//*[@id="Sva75c"]/div/div/div[3]/div[2]/c-wiz/div/div[1]/div[1]/div/div[2]/a/img').get_attribute('src'))
            # 로딩 후 다음 동작 실행, 최대 10초까지 대기
            driver.implicitly_wait(10)
            # 링크수집
            links.append(driver.find_element_by_xpath('//*[@id="Sva75c"]/div/div/div[3]/div[2]/c-wiz/div/div[1]/div[1]/div[3]/div/a/img').get_attribute('src'))
            driver.find_element_by_xpath('//*[@id="Sva75c"]/div/div/div[2]/a').click()
            #driver.find_element_by_xpath('//*[@id="Sva75c"]/div/div/div[2]/a').send_keys(Keys.ENTER)
            print(keyword+' 링크 수집 중..... number :'+str(i)+'/'+str(len(images)))
        except:
            continue

    forbidden=0
    for k,i in enumerate(links):
        try:
            url = i
            start = time.time()
            urllib.request.urlretrieve(url, "./"+keyword+"_high resolution/"+keyword+"_"+str(k-forbidden)+".jpg")
            print(str(k+1)+'/'+str(len(links))+' '+keyword+' 다운로드 중....... Download time : '+str(time.time() - start)[:5]+' 초')
        except:
            forbidden+=1
            continue
    print(keyword+' ---다운로드 완료---')
    
    driver.close()
    
# =============================================================================
# 실행
# =============================================================================
if __name__=='__main__':
    pool = Pool(processes=4) # 동시에 4개의 프로세스를 사용합니다.(4개의 크롬창을 띄움)
    pool.map(image_download, keyword)

들판 검색
제주공항 검색
Seoul train station 검색
인천공항 검색
김포공항 검색
River 검색
핑크뮬리 검색
한강 검색
River 스크롤 중 .............
인천공항 스크롤 중 .............
들판 스크롤 중 .............
Seoul train station 스크롤 중 .............
김포공항 스크롤 중 .............
제주공항 스크롤 중 .............
핑크뮬리 스크롤 중 .............
한강 스크롤 중 .............
River 찾은 이미지 개수: 376
Seoul train station 찾은 이미지 개수: 385
김포공항 찾은 이미지 개수: 384
핑크뮬리 찾은 이미지 개수: 384
인천공항 찾은 이미지 개수: 386
들판 찾은 이미지 개수: 387
제주공항 찾은 이미지 개수: 389
한강 찾은 이미지 개수: 384
River 링크 수집 중..... number :1/376
Seoul train station 링크 수집 중..... number :1/385
김포공항 링크 수집 중..... number :1/384
핑크뮬리 링크 수집 중..... number :1/384
인천공항 링크 수집 중..... number :1/386
들판 링크 수집 중..... number :1/387
River 링크 수집 중..... number :2/376
제주공항 링크 수집 중..... number :1/389
한강 링크 수집 중..... number :1/384
Seoul train station 링크 수집 중..... number :2/385
김포공항 링크 수집 중..... number :2/384
핑크뮬리 링크 수집 중..... number :2/384
인천공항 링크 수집 중..... number :2/386
들판 링크 수집 중..... number :2/387
제주공항 링크 수집 중..... number :2/389
River 링크 수집 중..... number :3/376
한

들판 링크 수집 중..... number :29/387
Seoul train station 링크 수집 중..... number :30/385
제주공항 링크 수집 중..... number :30/389
인천공항 링크 수집 중..... number :30/386
River 링크 수집 중..... number :33/376
한강 링크 수집 중..... number :30/384
김포공항 링크 수집 중..... number :31/384
들판 링크 수집 중..... number :30/387
핑크뮬리 링크 수집 중..... number :30/384
Seoul train station 링크 수집 중..... number :31/385
인천공항 링크 수집 중..... number :31/386
River 링크 수집 중..... number :34/376
제주공항 링크 수집 중..... number :31/389
한강 링크 수집 중..... number :31/384
김포공항 링크 수집 중..... number :32/384
핑크뮬리 링크 수집 중..... number :31/384
들판 링크 수집 중..... number :31/387
Seoul train station 링크 수집 중..... number :32/385
River 링크 수집 중..... number :35/376
제주공항 링크 수집 중..... number :32/389
한강 링크 수집 중..... number :32/384
인천공항 링크 수집 중..... number :32/386
김포공항 링크 수집 중..... number :33/384
들판 링크 수집 중..... number :32/387
핑크뮬리 링크 수집 중..... number :32/384
River 링크 수집 중..... number :36/376
제주공항 링크 수집 중..... number :33/389
Seoul train station 링크 수집 중..... number :33/385
한강 링크 수집 중..... number :33

들판 링크 수집 중..... number :59/387
핑크뮬리 링크 수집 중..... number :59/384
Seoul train station 링크 수집 중..... number :60/385
김포공항 링크 수집 중..... number :62/384
인천공항 링크 수집 중..... number :60/386
제주공항 링크 수집 중..... number :61/389
한강 링크 수집 중..... number :60/384
River 링크 수집 중..... number :68/376
김포공항 링크 수집 중..... number :63/384
Seoul train station 링크 수집 중..... number :61/385
들판 링크 수집 중..... number :60/387
핑크뮬리 링크 수집 중..... number :60/384
한강 링크 수집 중..... number :61/384
제주공항 링크 수집 중..... number :62/389
인천공항 링크 수집 중..... number :61/386
River 링크 수집 중..... number :69/376
한강 링크 수집 중..... number :62/384
Seoul train station 링크 수집 중..... number :62/385
들판 링크 수집 중..... number :61/387
핑크뮬리 링크 수집 중..... number :61/384
제주공항 링크 수집 중..... number :63/389
김포공항 링크 수집 중..... number :64/384
인천공항 링크 수집 중..... number :62/386
River 링크 수집 중..... number :70/376
Seoul train station 링크 수집 중..... number :63/385
핑크뮬리 링크 수집 중..... number :62/384
제주공항 링크 수집 중..... number :64/389
한강 링크 수집 중..... number :63/384
들판 링크 수집 중..... number :62/

들판 링크 수집 중..... number :87/387
핑크뮬리 링크 수집 중..... number :88/384
River 링크 수집 중..... number :101/376
Seoul train station 링크 수집 중..... number :91/385
제주공항 링크 수집 중..... number :94/389
인천공항 링크 수집 중..... number :89/386
김포공항 링크 수집 중..... number :94/384
한강 링크 수집 중..... number :91/384
들판 링크 수집 중..... number :88/387
River 링크 수집 중..... number :102/376
Seoul train station 링크 수집 중..... number :92/385
핑크뮬리 링크 수집 중..... number :89/384
제주공항 링크 수집 중..... number :95/389
인천공항 링크 수집 중..... number :90/386
River 링크 수집 중..... number :103/376
한강 링크 수집 중..... number :92/384
들판 링크 수집 중..... number :89/387
김포공항 링크 수집 중..... number :95/384
Seoul train station 링크 수집 중..... number :93/385
핑크뮬리 링크 수집 중..... number :90/384
제주공항 링크 수집 중..... number :96/389
인천공항 링크 수집 중..... number :91/386
River 링크 수집 중..... number :104/376
들판 링크 수집 중..... number :90/387
김포공항 링크 수집 중..... number :96/384
한강 링크 수집 중..... number :93/384
Seoul train station 링크 수집 중..... number :94/385
제주공항 링크 수집 중..... number :97/389
핑크뮬리 링크 수집 중..... numb

River 링크 수집 중..... number :132/376
한강 링크 수집 중..... number :120/384
핑크뮬리 링크 수집 중..... number :118/384
인천공항 링크 수집 중..... number :119/386
들판 링크 수집 중..... number :118/387
Seoul train station 링크 수집 중..... number :122/385
River 링크 수집 중..... number :133/376
한강 링크 수집 중..... number :121/384
핑크뮬리 링크 수집 중..... number :119/384
김포공항 링크 수집 중..... number :123/384
제주공항 링크 수집 중..... number :124/389
들판 링크 수집 중..... number :119/387
Seoul train station 링크 수집 중..... number :123/385
River 링크 수집 중..... number :134/376
김포공항 링크 수집 중..... number :124/384
River 링크 수집 중..... number :135/376
제주공항 링크 수집 중..... number :125/389
Seoul train station 링크 수집 중..... number :124/385
들판 링크 수집 중..... number :120/387
River 링크 수집 중..... number :136/376
제주공항 링크 수집 중..... number :126/389
김포공항 링크 수집 중..... number :125/384
Seoul train station 링크 수집 중..... number :125/385
들판 링크 수집 중..... number :121/387
인천공항 링크 수집 중..... number :121/386
제주공항 링크 수집 중..... number :127/389
김포공항 링크 수집 중..... number :126/384
Seoul train station 링크 수집 중..

...

333/361 사파리 다운로드 중....... Download time : 0.032 초


  File "/Users/sujin/opt/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/Users/sujin/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/Users/sujin/opt/anaconda3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/Users/sujin/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/Users/sujin/opt/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
KeyboardInterrupt


243/359 뮤지컬 공연 다운로드 중....... Download time : 0.164 초
334/361 사파리 다운로드 중....... Download time : 0.335 초
133/306 korean subway station entrance 다운로드 중....... Download time : 0.214 초
244/359 뮤지컬 공연 다운로드 중....... Download time : 0.352 초
297/353 Theme park 다운로드 중....... Download time : 0.396 초
336/361 사파리 다운로드 중....... Download time : 0.120 초
340/361 사파리 다운로드 중....... Download time : 0.101 초
249/359 뮤지컬 공연 다운로드 중....... Download time : 0.252 초
341/361 사파리 다운로드 중....... Download time : 0.180 초
250/359 뮤지컬 공연 다운로드 중....... Download time : 0.172 초
251/359 뮤지컬 공연 다운로드 중....... Download time : 0.058 초
342/361 사파리 다운로드 중....... Download time : 0.230 초
344/361 사파리 다운로드 중....... Download time : 0.066 초
140/306 korean subway station entrance 다운로드 중....... Download time : 0.080 초
254/359 뮤지컬 공연 다운로드 중....... Download time : 0.168 초
141/306 korean subway station entrance 다운로드 중....... Download time : 0.165 초
255/359 뮤지컬 공연 다운로드 중....... Download time : 0.152 초
142/306 korean subway station entrance 다운

304/359 뮤지컬 공연 다운로드 중....... Download time : 0.315 초
184/306 korean subway station entrance 다운로드 중....... Download time : 0.196 초
305/359 뮤지컬 공연 다운로드 중....... Download time : 0.230 초
324/353 Theme park 다운로드 중....... Download time : 0.379 초
185/306 korean subway station entrance 다운로드 중....... Download time : 0.220 초
325/353 Theme park 다운로드 중....... Download time : 0.136 초
306/359 뮤지컬 공연 다운로드 중....... Download time : 0.239 초
186/306 korean subway station entrance 다운로드 중....... Download time : 0.261 초
307/359 뮤지컬 공연 다운로드 중....... Download time : 0.250 초
308/359 뮤지컬 공연 다운로드 중....... Download time : 0.047 초
187/306 korean subway station entrance 다운로드 중....... Download time : 0.199 초
188/306 korean subway station entrance 다운로드 중....... Download time : 0.216 초
309/359 뮤지컬 공연 다운로드 중....... Download time : 0.358 초
189/306 korean subway station entrance 다운로드 중....... Download time : 0.294 초
310/359 뮤지컬 공연 다운로드 중....... Download time : 0.338 초
312/359 뮤지컬 공연 다운로드 중....... Download time : 0.058 초


245/306 korean subway station entrance 다운로드 중....... Download time : 0.230 초
246/306 korean subway station entrance 다운로드 중....... Download time : 0.184 초
247/306 korean subway station entrance 다운로드 중....... Download time : 0.200 초
343/353 Theme park 다운로드 중....... Download time : 1.887 초
248/306 korean subway station entrance 다운로드 중....... Download time : 0.198 초
249/306 korean subway station entrance 다운로드 중....... Download time : 0.222 초
250/306 korean subway station entrance 다운로드 중....... Download time : 0.261 초
251/306 korean subway station entrance 다운로드 중....... Download time : 0.229 초
252/306 korean subway station entrance 다운로드 중....... Download time : 0.295 초
253/306 korean subway station entrance 다운로드 중....... Download time : 0.266 초
344/353 Theme park 다운로드 중....... Download time : 1.473 초
254/306 korean subway station entrance 다운로드 중....... Download time : 0.356 초
255/306 korean subway station entrance 다운로드 중....... Download time : 0.300 초
256/306 korean subway station entrance 